In [26]:
from tensorflow.keras import Sequential
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.models import load_model
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import os

In [27]:
MAX_SEQ_LEN = 9

def sigmoid(x, with_noise=True):
    sig = 1 / (1 + np.exp(-x))
    if(with_noise):
        return sig + np.random.uniform(0,0.1, len(x))
    return sig

def sample_one_data(l, with_noise=True):
    start= -4.5
    x = np.linspace(start, start + l, l, False) 
    t = x + np.random.uniform(-0.1, 0.1, len(x))
    sigs = sigmoid(t, with_noise)
    return t, sigs

def sample_data(batch_size, l, with_noise=True):
    result = np.empty((batch_size, l))
    for i in range(batch_size):
        result[i,:] = sample_one_data(l, with_noise)[1]
    return result[:,:-1].reshape((batch_size, l-1, 1)), result[:, -1:]

class MyGenerator(Sequence):
    def __init__(self, batch_size=200, batch_num = MAX_SEQ_LEN - 1, shuffle=True):
        self.batch_size = batch_size
        self.batch_num = batch_num
        self.shuffle = shuffle
        
        # generate the samples
        self.batch_data = {}
        for i in range(2, 2+batch_num):
            X, y = sample_data(batch_size, i, with_noise=False)
            self.batch_data[i] = (X, y)
            
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.batch_num

    def __getitem__(self, index):
        return self.__data_generation(index)

    def on_epoch_end(self):
        'Shuffles indexes after each epoch'
        self.indexes = np.arange(self.batch_num)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        X, y = self.batch_data[index+2]
        return X, y

In [28]:
model_cp_path = 'model/lstm_sigmoid.hdf5'
def create_model():
    if(os.path.isfile(model_cp_path)):
       print("load existing model")
       model = load_model(model_cp_path)
       print(model.summary())
       return model
    
    model = Sequential()
    model.add(LSTM(10, input_shape=(None, 1), return_sequences=True))
    model.add(LSTM(5, input_shape=(None, 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
    print(model.summary())
    return model
                      
cp = ModelCheckpoint(
    filepath = model_cp_path,
    monitor = 'loss',
    mode = 'min',
    save_best_only=True)
es = EarlyStopping(monitor='loss', patience=50)
def train_model(model):
    train_data = MyGenerator(batch_size = 90)
    val_data = MyGenerator(batch_size = 10)
    model.fit(
        train_data,
        validation_data = val_data,
        epochs = 500,
        callbacks=[cp, es])
model = create_model()


load existing model
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 10)          480       
                                                                 
 lstm_1 (LSTM)               (None, 5)                 320       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 806
Trainable params: 806
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
train_model(model)

In [29]:
def test_model(batch_size, l):
    testX, testY = sample_data(batch_size, l, with_noise=False)
    print(testX)
    yhat = model.predict(testX)
    print((yhat-testY)/testY)

In [10]:
test_model(4, 8)

[[[0.01174852]
  [0.02773328]
  [0.07146586]
  [0.18935028]
  [0.38594487]
  [0.60926625]
  [0.80328379]]

 [[0.01139348]
  [0.02702191]
  [0.07827758]
  [0.17314163]
  [0.37621499]
  [0.63238909]
  [0.80668449]]

 [[0.01007294]
  [0.02836821]
  [0.0825333 ]
  [0.18834553]
  [0.36823949]
  [0.64273416]
  [0.81286598]]

 [[0.01186893]
  [0.0270533 ]
  [0.08169694]
  [0.19538236]
  [0.37531956]
  [0.64177073]
  [0.82280018]]]
[[-0.00221594]
 [ 0.00739721]
 [ 0.0004099 ]
 [ 0.00184962]]


In [24]:
def test_model_iter(batch_size, start_l):
    testX, target = sample_data(batch_size, MAX_SEQ_LEN, with_noise=False)
    print(testX)
    # start from the first start_l seq:
    testX = testX[:, :start_l-1]
    l = start_l - 1
    while(l < MAX_SEQ_LEN):
        print("evaluate textX:")
        print(testX)
        yhat = model.predict(testX)
        testX = np.concatenate((testX.reshape((batch_size, l)), yhat), axis=1)
        l += 1
       
    print((yhat-target)/target)

In [25]:
test_model_iter(4, 4)

[[[0.01154905]
  [0.02788961]
  [0.08139759]
  [0.18598707]
  [0.38596484]
  [0.6188432 ]
  [0.81786093]
  [0.9209393 ]]

 [[0.01079502]
  [0.03080518]
  [0.0746893 ]
  [0.18596632]
  [0.39374002]
  [0.61978698]
  [0.83101648]
  [0.92166735]]

 [[0.01052324]
  [0.03137689]
  [0.06994062]
  [0.18308916]
  [0.37470314]
  [0.63747598]
  [0.82552157]
  [0.9235981 ]]

 [[0.01078883]
  [0.02795538]
  [0.08054654]
  [0.17055414]
  [0.3919028 ]
  [0.62882056]
  [0.80706729]
  [0.91740551]]]
evaluate textX:
[[[0.01154905]
  [0.02788961]
  [0.08139759]]

 [[0.01079502]
  [0.03080518]
  [0.0746893 ]]

 [[0.01052324]
  [0.03137689]
  [0.06994062]]

 [[0.01078883]
  [0.02795538]
  [0.08054654]]]
evaluate textX:
[[0.01154905 0.02788961 0.08139759 0.18080011]
 [0.01079502 0.03080518 0.0746893  0.18074781]
 [0.01052324 0.03137689 0.06994062 0.18069509]
 [0.01078883 0.02795538 0.08054654 0.18071991]]
evaluate textX:
[[0.01154905 0.02788961 0.08139759 0.18080011 0.37802315]
 [0.01079502 0.03080518 0.074